# **Études de cooccurrences de mots**
Auteur : PM Nugues

Une petite analyse des cooccurrences des racines *priv* et *publi[ck]* dans les débats du riksdag au XVI<sup>e</sup>, XVII<sup>e</sup> et XVIII<sup>e</sup> siècles. On peut appliquer ce programme à n'importe quelle séquence de caractères. Il suffit de changer quelques paramètres.

# Préliminaires

Les modules nécessaires

In [1]:
import operator
import regex as re
import sys
from math import log, sqrt
from bs4 import BeautifulSoup
from tqdm import tqdm
from os import listdir

# Lecture des fichers XML et extraction des textes

## Liste des fichiers

On extrait la liste des fichers de débats. Les fichiers sont rassemblés pas décénnie.

In [2]:
dossiers = ['1520-40', '1540-60', '1560-80', '1580-1600','1600-1620', '1620-40', '1640-60', '1660-80', '1680-1700',
           '1700-1720', '1720-40']
dossiers = ['Riksdagstryck/' + dossier + '/' for dossier in dossiers]
dossiers

['Riksdagstryck/1520-40/',
 'Riksdagstryck/1540-60/',
 'Riksdagstryck/1560-80/',
 'Riksdagstryck/1580-1600/',
 'Riksdagstryck/1600-1620/',
 'Riksdagstryck/1620-40/',
 'Riksdagstryck/1640-60/',
 'Riksdagstryck/1660-80/',
 'Riksdagstryck/1680-1700/',
 'Riksdagstryck/1700-1720/',
 'Riksdagstryck/1720-40/']

In [3]:
fichiers_par_décénnie = [listdir(dossier) for dossier in dossiers]
fichiers_par_décénnie2 = []
fichiers_tous = []
for i in range(len(dossiers)):
    fichiers_par_décénnie2 += [[dossiers[i] + fichiers for fichiers in fichiers_par_décénnie[i]]]
    fichiers_tous += [dossiers[i] + fichiers for fichiers in fichiers_par_décénnie[i]]
fichiers_par_décénnie2

[['Riksdagstryck/1520-40/rda_1521-1560___01.xml'],
 ['Riksdagstryck/1540-60/rda_1521-1560___02.xml'],
 ['Riksdagstryck/1560-80/rda_1561-1592___01.xml'],
 ['Riksdagstryck/1580-1600/rda_1561-1592___02.xml',
  'Riksdagstryck/1580-1600/rda_1592-1597_3-2_.xml',
  'Riksdagstryck/1580-1600/rda_1597-1598___01.xml',
  'Riksdagstryck/1580-1600/rda_1597-1598___02.xml',
  'Riksdagstryck/1580-1600/rda_1592-1597_3-1_.xml'],
 ['Riksdagstryck/1600-1620/rda_1617_1_.xml',
  'Riksdagstryck/1600-1620/rda_1611-1616__.xml',
  'Riksdagstryck/1600-1620/rda_1617_2_.xml'],
 ['Riksdagstryck/1620-40/roa_1633-1638_2uppl_.xml',
  'Riksdagstryck/1620-40/roa_1627-1632__.xml',
  'Riksdagstryck/1620-40/roa_1633-1638_1uppl_.xml',
  'Riksdagstryck/1620-40/roa_1627-1632_2uppl_.xml'],
 ['Riksdagstryck/1640-60/roa_1660_1_.xml',
  'Riksdagstryck/1640-60/roa_1652-1655___02.xml',
  'Riksdagstryck/1640-60/roa_1652-1655___01.xml',
  'Riksdagstryck/1640-60/roa_1640-1644_2uppl_.xml',
  'Riksdagstryck/1640-60/pr_1642-1660___01.xml'

## Lecture des fichiers

On fait l'analyse syntaxique du code XML de chaque fichier

In [4]:
soups = []
for fichiers in fichiers_par_décénnie2:
    mini_soup = []
    for fichier in fichiers:
        print(fichier)
        with open(fichier, 'r', encoding='utf8') as infile:
            corpus_xml = infile.read()[1:]
        soup = BeautifulSoup(corpus_xml, 'xml')
        mini_soup += soup
    soups += [mini_soup]

Riksdagstryck/1520-40/rda_1521-1560___01.xml
Riksdagstryck/1540-60/rda_1521-1560___02.xml
Riksdagstryck/1560-80/rda_1561-1592___01.xml
Riksdagstryck/1580-1600/rda_1561-1592___02.xml
Riksdagstryck/1580-1600/rda_1592-1597_3-2_.xml
Riksdagstryck/1580-1600/rda_1597-1598___01.xml
Riksdagstryck/1580-1600/rda_1597-1598___02.xml
Riksdagstryck/1580-1600/rda_1592-1597_3-1_.xml
Riksdagstryck/1600-1620/rda_1617_1_.xml
Riksdagstryck/1600-1620/rda_1611-1616__.xml
Riksdagstryck/1600-1620/rda_1617_2_.xml
Riksdagstryck/1620-40/roa_1633-1638_2uppl_.xml
Riksdagstryck/1620-40/roa_1627-1632__.xml
Riksdagstryck/1620-40/roa_1633-1638_1uppl_.xml
Riksdagstryck/1620-40/roa_1627-1632_2uppl_.xml
Riksdagstryck/1640-60/roa_1660_1_.xml
Riksdagstryck/1640-60/roa_1652-1655___02.xml
Riksdagstryck/1640-60/roa_1652-1655___01.xml
Riksdagstryck/1640-60/roa_1640-1644_2uppl_.xml
Riksdagstryck/1640-60/pr_1642-1660___01.xml
Riksdagstryck/1640-60/roa_1645-1651__.xml
Riksdagstryck/1640-60/pr_1642-1660___02.xml
Riksdagstryck/1640

On vérifie le nombre de décénnies

In [5]:
len(soups)

11

On concatène les lignes de chaque fichier de sorte qu'il n'y ait qu'une ligne de texte par fichier

In [6]:
textes = []
cnt = 0
print('Nom du fichier et nombre de lignes')
for mini_soups in soups:
    textes_par_décénnie = []
    for soup in mini_soups:
        lines = soup.find_all('line')
        print(fichiers_tous[cnt], ':', len(lines))
        cnt += 1
        texte = ''
        for line in lines:
            texte = texte + line.get_text() + '\n'
        #textes_par_décénnie += texte
        textes_par_décénnie += [texte]
    textes += [textes_par_décénnie]

Nom du fichier et nombre de lignes
Riksdagstryck/1520-40/rda_1521-1560___01.xml : 27197
Riksdagstryck/1540-60/rda_1521-1560___02.xml : 12517
Riksdagstryck/1560-80/rda_1561-1592___01.xml : 29639
Riksdagstryck/1580-1600/rda_1561-1592___02.xml : 28356
Riksdagstryck/1580-1600/rda_1592-1597_3-2_.xml : 25162
Riksdagstryck/1580-1600/rda_1597-1598___01.xml : 28272
Riksdagstryck/1580-1600/rda_1597-1598___02.xml : 19203
Riksdagstryck/1580-1600/rda_1592-1597_3-1_.xml : 27741
Riksdagstryck/1600-1620/rda_1617_1_.xml : 11009
Riksdagstryck/1600-1620/rda_1611-1616__.xml : 25737
Riksdagstryck/1600-1620/rda_1617_2_.xml : 6102
Riksdagstryck/1620-40/roa_1633-1638_2uppl_.xml : 15164
Riksdagstryck/1620-40/roa_1627-1632__.xml : 11108
Riksdagstryck/1620-40/roa_1633-1638_1uppl_.xml : 12337
Riksdagstryck/1620-40/roa_1627-1632_2uppl_.xml : 9640
Riksdagstryck/1640-60/roa_1660_1_.xml : 10276
Riksdagstryck/1640-60/roa_1652-1655___02.xml : 5791
Riksdagstryck/1640-60/roa_1652-1655___01.xml : 24470
Riksdagstryck/1640-

On vérifie le nombre de textes

In [7]:
len(textes)

11

Nombre de textes par décénnie

In [8]:
list(map(len, textes))

[1, 1, 1, 5, 3, 4, 9, 9, 5, 9, 27]

# Concordances

Les séquences de caractères à rechercher et le contexte de la concordance

On fixe le choix à *priv* et *publi[ck]*. On peut les remplacer par n'importe quelle séquence

In [9]:
PRIVAT = True

In [10]:
if PRIVAT:
    chaîne = 'privat'
else:
    chaîne = 'publi[kc]'
contexte = 25

In [11]:
chaîne = re.sub(' ', '\\s+', chaîne)

In [12]:
for i, décénnie in enumerate(textes):
    print('===DÉCÉNNIE===', i + 1)
    for i, texte in enumerate(décénnie):
        print(fichiers[i], ':')
        texte = re.sub('\s+', ' ', texte.lower())
        concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
                       .format(pattern=chaîne, width=contexte))
        for match in re.finditer(concordance, texte):
            print('  ', match.group(1))

===DÉCÉNNIE=== 1
Riksdagstryck/1720-40/pr_1723___02.xml :
   en af uppteckningar till privat bruk. herredag i arboga 
   åger. 1544 423 danmarks »privaten sachen», sveriges intr
   holstain tayls auch alse privaten sachen, so unns unnd u
   ract alse dennemarkische privaten fur sich unnd unns nit
    och icke i konge. ma:tz privatt saker allenest utskicke
===DÉCÉNNIE=== 2
Riksdagstryck/1720-40/pr_1723___02.xml :
   - moriall på någre andre privat artichler». — en gammal 
   l till k. erik på några »privatartiklar» 805. > varén. *
===DÉCÉNNIE=== 3
Riksdagstryck/1720-40/pr_1723___02.xml :
   len, then skall man icke privatam aut auralem confession
   astore in posterum poena privationis. precationes et lit
   å offte både publice och privatim både för k. matt sielf
   orum; 14) de confessione privata, quomodo ea in abusum v
   llem, me in pristinam et privatam vitam redigi gauderem.
   scriptura? iii. utrum ex privati alicuius judicio aut ex
   riptura? iii. utrum [ex] privati alicuius j

# Cooccurrences

## Mots vides

On définit une liste de mots vides. On ne l'utilise pas dans le reste du programme

In [13]:
mots_vides = ['och', 'den', 'också', 'för']

## Fonctions de comptage des mots et des bigrammes

On définit les fonctions de comptage. Elles sont tirées de l'édition à venir de mon livre.

In [14]:
def découpe_mots(texte):
    mots = re.findall("\p{L}+|\p{N}+", texte)
    return mots


"""def découpe_mots(texte):
    mots = re.split('[\s\-,;:!?.’\'«»()–...&‘’“”*—]+', texte)
    mots.remove('')
    return mots"""


def compte_mots(mots):
    c_mots = {}
    for mot in mots:
        c_mots[mot] = c_mots.get(mot, 0) + 1
    return c_mots


def compte_bigrammes(mots):
    c_bigrammes = {}
    for i in range(len(mots) - 1):
        bigramme = (mots[i], mots[i + 1])
        c_bigrammes[bigramme] = c_bigrammes.get(bigramme, 0) + 1
    return c_bigrammes


def compte_bigrammes_fen(mots, fen=5):
    c_bigrammes_fen = {}
    for inx in range(1, fen + 1):
        for i in range(len(mots) - inx):
            bigramme = (mots[i], mots[i + inx])
            c_bigrammes_fen[bigramme] = c_bigrammes_fen.get(bigramme, 0) + 1
    return c_bigrammes_fen

On concatène les textes par décénnie. Pour chaque décénnie, on aura donc une ligne de texte

In [15]:
textes = [' '.join(décénnie) for décénnie in textes]

## Les mots

On compte les mots

In [16]:
c_mots_ordonnés_par_corpus = []
c_mots_par_corpus = []
textes_découpés = []
total_mots_par_corpus = []
for i, texte in enumerate(textes):
    #print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    texte = texte.lower()
    texte_découpé = découpe_mots(texte)
    total_mots_par_corpus += [len(texte_découpé)]
    textes_découpés += [texte_découpé]
    # print(mots)
    c_mots = compte_mots(texte_découpé)
    c_mots_par_corpus += [c_mots]
    c_mots_ordonnés = sorted(c_mots.items(), key=operator.itemgetter(1), reverse=True)
    c_mots_ordonnés_par_corpus += [c_mots_ordonnés]
    print('  ', c_mots_ordonnés[:10])
print('Nombre total de mots:', total_mots_par_corpus)

===DÉCÉNNIE=== 1
   [('och', 13902), ('i', 5621), ('som', 3428), ('till', 2798), ('att', 2643), ('the', 2380), ('af', 1982), ('så', 1835), ('ther', 1780), ('eller', 1777)]
===DÉCÉNNIE=== 2
   [('och', 6741), ('i', 2277), ('till', 1908), ('som', 1459), ('att', 1272), ('af', 1218), ('för', 869), ('så', 860), ('vi', 841), ('på', 829)]
===DÉCÉNNIE=== 3
   [('och', 12809), ('i', 6603), ('till', 3317), ('som', 3238), ('att', 3191), ('the', 2176), ('af', 2130), ('så', 1984), ('vi', 1710), ('för', 1664)]
===DÉCÉNNIE=== 4
   [('och', 65349), ('i', 24590), ('till', 17900), ('att', 16727), ('som', 16420), ('h', 12905), ('k', 12051), ('m', 10188), ('n', 9850), ('så', 9496)]
===DÉCÉNNIE=== 5
   [('och', 24517), ('i', 7100), ('att', 6696), ('till', 6019), ('som', 5668), ('m', 5029), ('h', 4675), ('k', 4380), ('t', 4137), ('så', 3631)]
===DÉCÉNNIE=== 6
   [('och', 19711), ('att', 9516), ('till', 6020), ('som', 4446), ('dhe', 4305), ('i', 4174), ('den', 3646), ('af', 3354), ('på', 3345), ('för', 2851)

On filtre les mots qui contiennent la séquence de caractères

In [17]:
occurrences_chaîne = []
for i, c_mots_ordonnés in enumerate(c_mots_ordonnés_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    #print(fichiers[i], ':')
    occurrences_chaîne_1 = list(filter(lambda x: re.search(chaîne, x[0]), 
                      c_mots_ordonnés))
    occurrences_chaîne += [occurrences_chaîne_1]
    print(' ', list(filter(lambda x: re.search(chaîne, x[0]), 
                      c_mots_ordonnés)))

===DÉCÉNNIE=== 1
  [('privaten', 3), ('privat', 1), ('privatt', 1)]
===DÉCÉNNIE=== 2
  [('privat', 1), ('privatartiklar', 1)]
===DÉCÉNNIE=== 3
  [('privatam', 2), ('privationis', 2), ('privati', 2), ('privatim', 1), ('privata', 1), ('privatis', 1), ('privatum', 1)]
===DÉCÉNNIE=== 4
  [('privat', 11), ('privatus', 5), ('privatis', 4), ('privatim', 2), ('privata', 2), ('privationis', 1), ('privatpersoner', 1), ('privatper', 1), ('privatbrev', 1), ('privato', 1), ('privations', 1), ('privatas', 1), ('privatce', 1), ('privatam', 1)]
===DÉCÉNNIE=== 5
  [('privat', 13), ('privatim', 2), ('privatz', 2), ('privatpersoner', 2), ('private', 1), ('privatsaker', 1), ('privationis', 1), ('privatisk', 1)]
===DÉCÉNNIE=== 6
  [('privat', 15), ('privatpersoner', 2), ('privatorum', 2), ('privatim', 2)]
===DÉCÉNNIE=== 7
  [('privat', 43), ('privati', 8), ('privatorum', 6), ('privatim', 6), ('privato', 6), ('privatum', 5), ('privatos', 5), ('privatis', 5), ('private', 4), ('privats', 4), ('privata', 4), (

### Fréquence absolue de la séquence

In [18]:
occurrences_chaîne

[[('privaten', 3), ('privat', 1), ('privatt', 1)],
 [('privat', 1), ('privatartiklar', 1)],
 [('privatam', 2),
  ('privationis', 2),
  ('privati', 2),
  ('privatim', 1),
  ('privata', 1),
  ('privatis', 1),
  ('privatum', 1)],
 [('privat', 11),
  ('privatus', 5),
  ('privatis', 4),
  ('privatim', 2),
  ('privata', 2),
  ('privationis', 1),
  ('privatpersoner', 1),
  ('privatper', 1),
  ('privatbrev', 1),
  ('privato', 1),
  ('privations', 1),
  ('privatas', 1),
  ('privatce', 1),
  ('privatam', 1)],
 [('privat', 13),
  ('privatim', 2),
  ('privatz', 2),
  ('privatpersoner', 2),
  ('private', 1),
  ('privatsaker', 1),
  ('privationis', 1),
  ('privatisk', 1)],
 [('privat', 15), ('privatpersoner', 2), ('privatorum', 2), ('privatim', 2)],
 [('privat', 43),
  ('privati', 8),
  ('privatorum', 6),
  ('privatim', 6),
  ('privato', 6),
  ('privatum', 5),
  ('privatos', 5),
  ('privatis', 5),
  ('private', 4),
  ('privats', 4),
  ('privata', 4),
  ('privatus', 3),
  ('privation', 3),
  ('privat

### Fréquence relative de la séquence par décénnie

In [19]:
for i in range(len(total_mots_par_corpus)):
    print(sum([x[1] for x in occurrences_chaîne[i]])/total_mots_par_corpus[i])

2.0161209027382953e-05
1.786352268667381e-05
3.9845876151047745e-05
2.765839038225572e-05
5.687902326352051e-05
5.776292445434666e-05
0.00010435355366931986
0.00016284718923812718
0.0002659944158333605
0.00011261166365730509
0.00011048186090092222


## Les bigrammes

On compte les bigrammes et on les ordonne par fréquence

In [20]:
c_bigrammes_ordonnés_par_corpus = []
c_bigrammes_par_corpus = []
for i, texte_découpé in enumerate(textes_découpés):
    print('===DÉCÉNNIE===', i + 1)
    # print(fichiers[i], ':')
    c_bigrammes = compte_bigrammes(texte_découpé)
    c_bigrammes_par_corpus += [c_bigrammes]
    c_bigrammes_ordonnés = sorted(c_bigrammes.items(), 
                                  key=operator.itemgetter(1), reverse=True)
    c_bigrammes_ordonnés_par_corpus += [c_bigrammes_ordonnés]
    print('  ', c_bigrammes_ordonnés[:15])

===DÉCÉNNIE=== 1
   [(('k', 'm'), 475), (('m', 't'), 450), (('hans', 'nåde'), 329), (('h', 'k'), 304), (('ma', 't'), 271), (('att', 'the'), 235), (('och', 'så'), 231), (('i', 's'), 191), (('hans', 'nade'), 183), (('som', 'i'), 174), (('som', 'the'), 171), (('ath', 'the'), 165), (('och', 'en'), 160), (('hvar', 'och'), 152), (('och', 'i'), 150)]
===DÉCÉNNIE=== 2
   [(('k', 'm'), 227), (('m', 't'), 149), (('k', 'gustafs'), 143), (('bref', 'till'), 142), (('ma', 't'), 133), (('kong', 'e'), 125), (('till', 'att'), 118), (('gustafs', 'bref'), 112), (('h', 'k'), 109), (('och', 'så'), 106), (('vi', 'och'), 106), (('som', 'vi'), 98), (('så', 'att'), 95), (('och', 'hans'), 94), (('högbe', 'te'), 93)]
===DÉCÉNNIE=== 3
   [(('k', 'm'), 553), (('h', 'k'), 428), (('kong', 'e'), 408), (('m', 't'), 345), (('att', 'the'), 333), (('till', 'att'), 271), (('e', 'k'), 251), (('ma', 't'), 219), (('e', 'ma'), 212), (('ther', 'till'), 207), (('att', 'vi'), 186), (('m', 'tz'), 184), (('så', 'och'), 181), (('m'

Les bigrammes qui contienne la séquence de caractères

In [21]:
for i, c_bigrammes_ordonnés in enumerate(c_bigrammes_ordonnés_par_corpus):
    # print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    print(' ', list(filter(lambda x: re.search(chaîne, x[0][0]) or 
                           re.search(chaîne, x[0][1]), 
                      c_bigrammes_ordonnés)))

===DÉCÉNNIE=== 1
  [(('privaten', 'sachen'), 2), (('till', 'privat'), 1), (('privat', 'bruk'), 1), (('danmarks', 'privaten'), 1), (('alse', 'privaten'), 1), (('dennemarkische', 'privaten'), 1), (('privaten', 'fur'), 1), (('tz', 'privatt'), 1), (('privatt', 'saker'), 1)]
===DÉCÉNNIE=== 2
  [(('andre', 'privat'), 1), (('privat', 'artichler'), 1), (('några', 'privatartiklar'), 1), (('privatartiklar', '805'), 1)]
===DÉCÉNNIE=== 3
  [(('poena', 'privationis'), 2), (('ex', 'privati'), 2), (('privati', 'alicuius'), 2), (('icke', 'privatam'), 1), (('privatam', 'aut'), 1), (('privationis', 'precationes'), 1), (('och', 'privatim'), 1), (('privatim', 'både'), 1), (('confessione', 'privata'), 1), (('privata', 'quomodo'), 1), (('et', 'privatam'), 1), (('privatam', 'vitam'), 1), (('ex', 'privatis'), 1), (('privatis', 'alicuius'), 1), (('tam', 'privatum'), 1), (('privatum', 'quarn'), 1), (('privationis', 'ut'), 1)]
===DÉCÉNNIE=== 4
  [(('privat', 'sak'), 3), (('beneficio', 'privatus'), 3), (('privat'

Les bigrammes avec une fenêtre. On considère maintenant le contexte à droite et à gauche du mot

Les bigrammes les plus fréquents

In [22]:
c_bigrammes_fen_ordonnés_par_corpus = []
for i, texte_découpé in enumerate(textes_découpés):
    # print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    c_bigrammes_fen = compte_bigrammes_fen(texte_découpé)
    c_bigrammes_fen_ordonnés = sorted(c_bigrammes_fen.items(), 
                              key=operator.itemgetter(1), reverse=True)
    c_bigrammes_fen_ordonnés_par_corpus += [c_bigrammes_fen_ordonnés]
    print(c_bigrammes_fen_ordonnés[:10])

===DÉCÉNNIE=== 1
[(('och', 'och'), 3309), (('och', 'i'), 1285), (('och', 'som'), 1135), (('i', 'och'), 1110), (('och', 'att'), 955), (('som', 'och'), 810), (('till', 'och'), 798), (('och', 'så'), 775), (('och', 'the'), 755), (('och', 'till'), 744)]
===DÉCÉNNIE=== 2
[(('och', 'och'), 1616), (('och', 'som'), 560), (('till', 'och'), 527), (('och', 'till'), 470), (('och', 'så'), 410), (('som', 'och'), 396), (('och', 'i'), 391), (('och', 'att'), 382), (('i', 'och'), 363), (('och', 'ther'), 351)]
===DÉCÉNNIE=== 3
[(('och', 'och'), 2802), (('i', 'i'), 2256), (('och', 'som'), 1015), (('och', 'att'), 989), (('och', 'i'), 946), (('i', 'och'), 846), (('och', 'till'), 842), (('till', 'och'), 828), (('och', 'så'), 780), (('som', 'och'), 722)]
===DÉCÉNNIE=== 4
[(('och', 'och'), 14958), (('k', 'm'), 8598), (('f', 'n'), 7322), (('och', 'i'), 5818), (('h', 'f'), 5461), (('till', 'och'), 5401), (('och', 'som'), 5346), (('och', 'att'), 5264), (('h', 'n'), 5132), (('h', 'k'), 5056)]
===DÉCÉNNIE=== 5
[(('o

Les bigrammes avec la séquence de caractères recherchée

In [23]:
for i, c_bigrammes_fen_ordonnés in enumerate(c_bigrammes_fen_ordonnés_par_corpus):
    #print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or 
                      re.search(chaîne, x[0][1]), 
                      c_bigrammes_fen_ordonnés)))

===DÉCÉNNIE=== 1
[(('privaten', 'sachen'), 2), (('alse', 'privaten'), 2), (('privaten', 'unns'), 2), (('privaten', 'unnd'), 2), (('till', 'privat'), 1), (('privat', 'bruk'), 1), (('danmarks', 'privaten'), 1), (('dennemarkische', 'privaten'), 1), (('privaten', 'fur'), 1), (('tz', 'privatt'), 1), (('privatt', 'saker'), 1), (('uppteckningar', 'privat'), 1), (('privat', 'herredag'), 1), (('423', 'privaten'), 1), (('privaten', 'sveriges'), 1), (('auch', 'privaten'), 1), (('privaten', 'so'), 1), (('privaten', 'sich'), 1), (('ma', 'privatt'), 1), (('privatt', 'allenest'), 1), (('af', 'privat'), 1), (('privat', 'i'), 1), (('1544', 'privaten'), 1), (('privaten', 'intressen'), 1), (('tayls', 'privaten'), 1), (('contract', 'privaten'), 1), (('konge', 'privatt'), 1), (('privatt', 'utskicket'), 1), (('karaktären', 'privat'), 1), (('privat', 'arboga'), 1), (('svåger', 'privaten'), 1), (('privaten', 'ej'), 1), (('holstain', 'privaten'), 1), (('vorigen', 'privaten'), 1), (('i', 'privatt'), 1), (('priv

# Mesures de cooccurrence

On applique maintenant trois mesures de cooccurrence:
* l'information mutuelle de Fano
* les t-scores
* le rapport de vraisemblance

## Information mutuelle

Définition de l'information mutuelle

In [24]:
def info_mutuelle(c_mots, c_bigrammes, taille):
    c_info_mutuelle = {}
    for bigramme in c_bigrammes.keys():
        c_info_mutuelle[bigramme] = log(taille * c_bigrammes[bigramme] / 
                                        (c_mots[bigramme[0]] * c_mots[bigramme[1]]), 2.0)
    return c_info_mutuelle

On ordonne les bigrammes par valeur d'information mutuelle décroissante

In [25]:
c_info_mutuelle_ordonnée_par_corpus = []
c_info_mutuelle_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_info_mutuelle = info_mutuelle(c_mots, c_bigrammes, len(texte_découpé))
    c_info_mutuelle_par_corpus += [c_info_mutuelle]
    c_info_mutuelle_ordonnée = sorted(c_info_mutuelle.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_info_mutuelle_ordonnée[:10])
    c_info_mutuelle_ordonnée_par_corpus += [c_info_mutuelle_ordonnée]

===DÉCÉNNIE=== 1
   [(('ehva', 'rfv'), 17.91998641235595), (('konseqvent', 'anlita'), 17.91998641235595), (('anlita', 'tryckpressen'), 17.91998641235595), (('lagens', 'revision'), 17.91998641235595), (('liturgien', '1577'), 17.91998641235595), (('confessio', 'fidei'), 17.91998641235595), (('fidei', '1593'), 17.91998641235595), (('1726', '4727'), 17.91998641235595), (('1728', '1733'), 17.91998641235595), (('1733', 'utgaf'), 17.91998641235595)]
===DÉCÉNNIE=== 2
   [(('kop', 'perbergs'), 16.77262386646028), (('listeligen', 'trachtede'), 16.77262386646028), (('bätter', 'tilförlatt'), 16.77262386646028), (('olämpe', 'tilskvnde'), 16.77262386646028), (('forrätlige', 'hiärter'), 16.77262386646028), (('hiärter', 'ingifne'), 16.77262386646028), (('ohörsamhet', 'gemenligen'), 16.77262386646028), (('unsacht', 'hotedt'), 16.77262386646028), (('pfaltzgref', 'viske'), 16.77262386646028), (('almänneligen', 'rychtes'), 16.77262386646028)]
===DÉCÉNNIE=== 3
   [(('tillsvidare', 'afbryta'), 17.9371381491

L'information mutuelle des mots plus fréquents avec un seuil. Le seuil est assez haut pour le pas avoir trop d'affichages

In [26]:
seuil = 200
for i, c_mots, c_bigrammes, c_info_mutuelle, c_info_mutuelle_ordonnée in zip(
    range(len(textes)),
    c_mots_par_corpus,
    c_bigrammes_par_corpus,
    c_info_mutuelle_par_corpus,
    c_info_mutuelle_ordonnée_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    for bigramme_im in c_info_mutuelle_ordonnée:
        if c_bigrammes[bigramme_im[0]] >= seuil:
            #mots_du_bigramme = bigramme_im[0].split()
            print('  ', c_info_mutuelle[bigramme_im[0]], "\t", 
                  bigramme_im[0], "\t", c_bigrammes[bigramme_im[0]], "\t",
                  c_mots[bigramme_im[0][0]], "\t", c_mots[bigramme_im[0][1]])

===DÉCÉNNIE=== 1
   7.821400192633897 	 ('h', 'k') 	 304 	 434 	 768
   7.651268368018571 	 ('k', 'm') 	 475 	 768 	 763
   7.5664977449239235 	 ('ma', 't') 	 271 	 408 	 869
   7.395015989924054 	 ('m', 't') 	 450 	 763 	 869
   7.004215984001913 	 ('hans', 'nåde') 	 329 	 1485 	 428
   3.2117967910840712 	 ('att', 'the') 	 235 	 2643 	 2380
   1.1671662764554518 	 ('och', 'så') 	 231 	 13902 	 1835
===DÉCÉNNIE=== 2
   6.294201680414793 	 ('k', 'm') 	 227 	 649 	 499
===DÉCÉNNIE=== 3
   7.830036487578805 	 ('kong', 'e') 	 408 	 462 	 974
   7.475185796216387 	 ('e', 'ma') 	 212 	 974 	 307
   7.324416892853389 	 ('ma', 't') 	 219 	 307 	 1117
   6.704669513043162 	 ('k', 'm') 	 553 	 1154 	 1153
   6.232382027647109 	 ('h', 'k') 	 428 	 1238 	 1154
   6.071000432636152 	 ('m', 't') 	 345 	 1153 	 1117
   5.8084762323286565 	 ('e', 'k') 	 251 	 974 	 1154
   3.589260781647319 	 ('att', 'the') 	 333 	 3191 	 2176
   3.3257830716750325 	 ('ther', 'till') 	 207 	 1562 	 3317
   2.68383100

Les bigrammes avec la séquence qu'on recherche ordonnés par information mutuelle 

In [27]:
for i, corpus in enumerate(c_info_mutuelle_ordonnée_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

===DÉCÉNNIE=== 1
[(('dennemarkische', 'privaten'), 16.335023911634792), (('privat', 'bruk'), 15.33502391163479), (('privaten', 'sachen'), 13.6345841934937), (('privaten', 'fur'), 13.16509891019248), (('danmarks', 'privaten'), 12.875592292997494), (('alse', 'privaten'), 12.527668989577187), (('tz', 'privatt'), 10.099807449940759), (('privatt', 'saker'), 10.001123175081352), (('till', 'privat'), 6.469806165191194)]
===DÉCÉNNIE=== 2
[(('privatartiklar', '805'), 14.772623866460279), (('privat', 'artichler'), 14.187661365739121), (('några', 'privatartiklar'), 10.128767676685552), (('andre', 'privat'), 8.744717869890392)]
===DÉCÉNNIE=== 3
[(('privata', 'quomodo'), 17.937138149105706), (('privationis', 'precationes'), 16.937138149105706), (('confessione', 'privata'), 16.937138149105706), (('privati', 'alicuius'), 16.352175648384552), (('privatis', 'alicuius'), 16.352175648384552), (('poena', 'privationis'), 15.352175648384552), (('privatam', 'vitam'), 15.352175648384552), (('tam', 'privatum')

## t-scores

Définition des t-scores

In [28]:
def t_scores(words, freq_unigrams, freq_bigrams):
    ts = {}
    for bigram in freq_bigrams:
        ts[bigram] = ((freq_bigrams[bigram] -
                      freq_unigrams[bigram[0]] *
                      freq_unigrams[bigram[1]] /
                      len(words)) /
                      sqrt(freq_bigrams[bigram]))
    return ts

On calcule les t-scores et on affiche les plus hauts

In [29]:
c_t_scores_ordonnée_par_corpus = []
c_t_scores_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    # print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_t_scores = t_scores(texte_découpé, c_mots, c_bigrammes)
    c_t_scores_par_corpus += [c_t_scores]
    c_t_scores_ordonnée = sorted(c_t_scores.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_t_scores_ordonnée[:10])
    c_t_scores_ordonnée_par_corpus += [c_t_scores_ordonnée]

===DÉCÉNNIE=== 1
   [(('k', 'm'), 21.686080679537064), (('m', 't'), 21.08717033359404), (('hans', 'nåde'), 17.997064734352747), (('h', 'k'), 17.358512398402247), (('ma', 't'), 16.375233224539944), (('att', 'the'), 13.675133577526216), (('hans', 'nade'), 13.430369171605783), (('i', 's'), 12.854315802699043), (('ath', 'the'), 11.994280160000711), (('gode', 'men'), 11.587086450551894)]
===DÉCÉNNIE=== 2
   [(('k', 'm'), 14.874533233532865), (('m', 't'), 12.03312006199931), (('k', 'gustafs'), 11.879247226209095), (('bref', 'till'), 11.488770561609524), (('ma', 't'), 11.462297721515135), (('kong', 'e'), 11.141686828183712), (('gustafs', 'bref'), 10.541872575177774), (('h', 'k'), 10.332592971694531), (('högbe', 'te'), 9.625890323606253), (('hans', 'kong'), 9.420878525876393)]
===DÉCÉNNIE=== 3
   [(('k', 'm'), 23.290499077995012), (('h', 'k'), 20.412999187239397), (('kong', 'e'), 20.110242327092337), (('m', 't'), 18.297891252925407), (('att', 'the'), 16.732120872246025), (('e', 'k'), 15.560288

Les bigrammes avec la séquence qu'on recherche ordonnés par t-scores 

In [30]:
for i, corpus in enumerate(c_t_scores_ordonnée_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

===DÉCÉNNIE=== 1
[(('privaten', 'sachen'), 1.4141023645776576), (('dennemarkische', 'privaten'), 0.9999879032745835), (('privat', 'bruk'), 0.9999758065491672), (('privaten', 'fur'), 0.9998911294712521), (('danmarks', 'privaten'), 0.9998669360204193), (('alse', 'privaten'), 0.99983064584417), (('tz', 'privatt'), 0.9990887133519623), (('privatt', 'saker'), 0.9990241974830747), (('till', 'privat'), 0.9887177874282765)]
===DÉCÉNNIE=== 2
[(('privatartiklar', '805'), 0.9999642729546266), (('privat', 'artichler'), 0.99994640943194), (('några', 'privatartiklar'), 0.9991068238656663), (('andre', 'privat'), 0.997668810289389)]
===DÉCÉNNIE=== 3
[(('privati', 'alicuius'), 1.4141966571995577), (('poena', 'privationis'), 1.4141797520260206), (('ex', 'privati'), 1.4137571226875896), (('privata', 'quomodo'), 0.9999960154123849), (('privationis', 'precationes'), 0.9999920308247698), (('confessione', 'privata'), 0.9999920308247698), (('privatis', 'alicuius'), 0.9999880462371546), (('privatam', 'vitam'),

## Log du rapport de vraisemblance (log-likelihood-ration)

Définition du rapport de vraisemblance

In [31]:
def likelihood_ratio(words, freq_unigrams, freq_bigrams):
    lr = {}
    for bigram in freq_bigrams:
        p = freq_unigrams[bigram[1]] / len(words)
        p1 = freq_bigrams[bigram] / freq_unigrams[bigram[0]]
        p2 = ((freq_unigrams[bigram[1]] - freq_bigrams[bigram])
              / (len(words) - freq_unigrams[bigram[0]]))
        if p1 != 1.0 and p2 != 0.0:
            lr[bigram] = 2.0 * (
                log_f(freq_bigrams[bigram],
                      freq_unigrams[bigram[0]], p1) +
                log_f(freq_unigrams[bigram[1]] -
                      freq_bigrams[bigram],
                      len(words) - freq_unigrams[bigram[0]], p2) -
                log_f(freq_bigrams[bigram],
                      freq_unigrams[bigram[0]], p) -
                log_f(freq_unigrams[bigram[1]] -
                      freq_bigrams[bigram],
                      len(words) - freq_unigrams[bigram[0]], p))
    return lr

In [32]:
def log_f(k, N, p):
    return k * log(p) + (N - k) * log(1 - p)

On le calcule

In [33]:
c_lr_ordonné_par_corpus = []
c_lr_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_lr = likelihood_ratio(texte_découpé, c_mots, c_bigrammes)
    c_lr_par_corpus += [c_lr]
    c_lr_ordonné = sorted(c_lr.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_lr_ordonné[:10])
    c_lr_ordonné_par_corpus += [c_lr_ordonné]

===DÉCÉNNIE=== 1
   [(('k', 'm'), 4862.17958257203), (('m', 't'), 4344.397098241381), (('h', 'k'), 3123.2581527287093), (('hans', 'nåde'), 2984.1378901637336), (('ma', 't'), 2638.9714342289317), (('anf', 'arb'), 1779.1857736267502), (('kon', 'ge'), 1710.1278863096165), (('hans', 'nade'), 1697.6726508316783), (('s', 'registratur'), 1356.1910188635861), (('rikets', 'råd'), 1281.9024819147244)]
===DÉCÉNNIE=== 2
   [(('k', 'm'), 1742.93324006075), (('k', 'gustafs'), 1385.7889723547237), (('kong', 'e'), 1347.846766005545), (('ma', 't'), 1260.0232167883069), (('högbe', 'te'), 1212.9883786689497), (('gustafs', 'bref'), 1173.2563167284861), (('m', 't'), 1073.8223953426295), (('h', 'k'), 876.9810075005362), (('bref', 'till'), 758.3858902441971), (('hans', 'kong'), 677.8645241628712)]
===DÉCÉNNIE=== 3
   [(('k', 'm'), 4679.59687870739), (('kong', 'e'), 4398.012843761489), (('h', 'k'), 3197.7149982837836), (('m', 't'), 2453.139965091086), (('ma', 't'), 2050.2640017170415), (('e', 'ma'), 2024.3617

Les bigrammes avec la séquence qu'on recherche ordonnés par rapports de vraisemblance

In [34]:
for i, corpus in enumerate(c_lr_ordonné_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

===DÉCÉNNIE=== 1
[(('privaten', 'sachen'), 35.930752510555266), (('privaten', 'fur'), 16.744438382459293), (('danmarks', 'privaten'), 16.321454225357577), (('alse', 'privaten'), 15.818574821321405)]
===DÉCÉNNIE=== 2
[]
===DÉCÉNNIE=== 3
[(('privatam', 'vitam'), 20.274491692659762), (('privatam', 'aut'), 15.131414517664894), (('privationis', 'ut'), 11.473693454211116), (('et', 'privatam'), 9.366974552674968), (('icke', 'privatam'), 7.965743035269785)]
===DÉCÉNNIE=== 4
[(('beneficio', 'privatus'), 69.90062601690926), (('privatis', 'injuriis'), 47.83150477514099), (('commoda', 'privatis'), 42.653244233685975), (('privat', 'sak'), 40.940164721607744), (('privatis', 'iniuriis'), 22.712912287447107), (('publico', 'privatis'), 20.481491860544452), (('eorum', 'privatim'), 20.160684831593855), (('privat', 'saker'), 18.717297340508594), (('bevarade', 'privata'), 17.963027834785073), (('privatus', 'officio'), 17.49892040052339), (('privatim', 'hoc'), 17.290079479120777), (('privat', 'natur'), 16.5